In [1]:
# v1
import numpy as np
import copy
import time as Time
import matplotlib.pyplot as plt
import pandas as pd

def compare(p1, p2):
    D = len(p1)
    p1_dominate_p2 = True
    p2_dominate_p1 = True
    for i in range(D):
        if p1[i] > p2[i]:
            p1_dominate_p2 = False
        if p1[i] < p2[i]:
            p2_dominate_p1 = False
    if p1_dominate_p2 == p2_dominate_p1:
        return 0
    return 1 if p1_dominate_p2 else -1
def fast_non_dominated_sort(P):
    P_size = len(P)
    n = np.full(shape=P_size, fill_value=0)
    S = []
    f = []
    rank = np.full(shape=P_size, fill_value=-1)
    f_0 = []
    for p in range(P_size):
        n_p = 0
        S_p = []
        for q in range(P_size):
            if p == q:
                continue
            cmp = compare(P[p], P[q])
            if cmp == 1:
                S_p.append(q)
            elif cmp == -1:
                n_p += 1
        S.append(S_p)
        n[p] = n_p
        if n_p == 0:
            rank[p] = 0
            f_0.append(p)
    f.append(f_0) 
    i = 0
    while len(f[i]) != 0:
        Q = []
        for p in f[i]:
            for q in S[p]:
                n[q] -= 1
                if n[q] == 0:
                    rank[q] = i + 1
                    Q.append(q)
        i += 1
        f.append(Q)
    rank +=1
    return rank,f

def crossover(pop1,pop2,num_of_part,constr_mat,alpha):
    pop_a = copy.deepcopy(pop1)
    pop_b = copy.deepcopy(pop2)
    cross_position = np.random.randint(low=0, high=num_of_part, size=(2))
    start = min(cross_position[0], cross_position[1])
    end = max(cross_position[0], cross_position[1])
    piece = []
    for i in range(num_of_part):
        if pop_b[i] in pop_a[start:end+1]:
            piece.append(pop_b[i])
    pop_a[start:end+1] = piece
    if np.random.uniform(0,1)<alpha:
        pop_a = mutation(pop_a, constr_mat, num_of_part)
    return pop_a
def mutation(pop_, constr_mat, num_of_part):
    pop_c = copy.deepcopy(pop_)
    pop_c = pop_c.tolist()
    mutation_position = np.random.randint(0, num_of_part)
    mutation_part = pop_c[mutation_position]
    tight_ahead_task = np.argwhere(constr_mat[:,mutation_part]==1).reshape(-1).tolist()
    tight_post_task = np.argwhere(constr_mat[mutation_part,:]==1).reshape(-1).tolist()
    
    if len(tight_ahead_task)!=0 and len(tight_post_task)!=0:
        pop_c.remove(mutation_part)
        for i in range(len(pop_c)):
            if pop_c[i] in tight_ahead_task:
                flagFront = i
        for i in range(len(pop_c)):
            if pop_c[i] in tight_post_task:
                flagBack = i
                break
        pop_c.insert(np.random.choice(range(flagFront+1,flagBack+1)), mutation_part)
    elif len(tight_ahead_task)!=0:
        pop_c.remove(mutation_part)
        for i in range(len(pop_c)):
            if pop_c[i] in tight_ahead_task:
                flagFront = i
        pop_c.insert(np.random.choice(range(flagFront+1,len(pop_c)+1)), mutation_part)
    elif len(tight_post_task)!=0:
        pop_c.remove(mutation_part)
        for i in range(len(pop_c)):
            if pop_c[i] in tight_post_task:
                flagBack = i
                break
        pop_c.insert(np.random.choice(range(0,flagBack+1)), mutation_part)
    else:
        pop_c.remove(mutation_part)
        pop_c.insert(np.random.choice(range(0,len(pop_c)+1)), mutation_part)
    pop_c = np.array(pop_c)
    return pop_c


def softmax(f):
    f -= np.max(f)
    return np.exp(f) / np.sum(np.exp(f))
def getChoiceProb(branchInfo, visitNumInfo, visitNumOfParent, temp, p):
    alpha_Hb = (visitNumInfo+0.000000000001)/(visitNumOfParent + 0.000000001)
    alpha_Hb = np.min(np.concatenate((alpha_Hb[np.newaxis,...], 
                                      np.ones(alpha_Hb.shape)[np.newaxis,...])), axis=0)
    beta_Hb = (visitNumInfo+0.000000000001)/(branchInfo + 0.000000001)
    beta_Hb = np.min(np.concatenate((beta_Hb[np.newaxis,...],
                                      np.ones(beta_Hb.shape)[np.newaxis,...])), axis=0)
    if np.random.uniform() >= p:
        logits = -1*np.log(beta_Hb)
    else:
        logits = -1*np.log(alpha_Hb)
    prob = softmax(logits/temp)
    return prob
class get_candid:
    def __init__(self, prior_constrain, disassembly_side):
        self.prior_constrain = copy.deepcopy(prior_constrain)
        self.disassembly_side = disassembly_side
    def get_can_do_task(self, did_tasks):
        did_tasks_with0_encoding = [abs(ele)-1 for ele in did_tasks]
        self.prior_constrain[did_tasks_with0_encoding, :] = 0
        can_do_tasks = np.argwhere(np.sum(self.prior_constrain, axis=0)==0).reshape(-1).tolist()
        for did_task in did_tasks_with0_encoding:
            can_do_tasks.remove(did_task)
        can_do_tasks = (np.array(can_do_tasks) + 1).tolist()
        can_do_tasks_duplic = copy.deepcopy(can_do_tasks)
        for ele in can_do_tasks_duplic:
            if self.disassembly_side[ele-1] == -1:
                can_do_tasks.remove(ele)
                can_do_tasks.append(-1*ele)
            elif self.disassembly_side[ele-1] == 0:
                can_do_tasks.append(-1*ele)
        return can_do_tasks

def init(num_ele, pop_num, temperature, prior_constrain, disassembly_side):
    # construct trees using dict
    global memory_visit
    global memory_branch
    memory_visit = dict()
    memory_branch = dict()
    pops = []
    pop_num = pop_num*2
    last_unique_num = len(pops)
    while len(pops)<pop_num:
        pop = []
        candidate = get_candid(prior_constrain, disassembly_side)
        candid_nodes = candidate.get_can_do_task(pop)
        candid_nodes = (np.array(candid_nodes)+num_ele).tolist()
        
        while len(pop) < num_ele: 
            layer = len(pop)
            if layer == 0:
                candid_chain = [pop+[ele-num_ele] for ele in candid_nodes]
                condid_brance_info = []
                condid_visit_info = []
                for candid_chain_ele in candid_chain:
                    candid_dic_key = ''.join(str(candid_chain_ele_ele) 
                                               for candid_chain_ele_ele in candid_chain_ele)
                    if memory_branch.get(candid_dic_key) != None:
                        condid_brance_info.append(memory_branch[candid_dic_key])
                    else:
                        condid_brance_info.append(0)
                    if memory_visit.get(candid_dic_key) != None:
                        condid_visit_info.append(memory_visit[candid_dic_key])
                    else:
                        condid_visit_info.append(0)
                condid_brance_info = np.array(condid_brance_info)
                condid_visit_info = np.array(condid_visit_info)
                parent_visit_info = []
                for candid_nodes_ele in candid_nodes:
                    parent_dic_key = str(candid_nodes_ele-num_ele)
                    if memory_visit.get(parent_dic_key) != None:
                        parent_visit_info.append(memory_visit[parent_dic_key])
                    else:
                        parent_visit_info.append(0)
                parent_visit_info = np.sum(parent_visit_info)
                choice_prob = getChoiceProb(condid_brance_info, condid_visit_info, parent_visit_info, 
                                            temperature, 1-len(pops)/pop_num)
                choiced_node = np.random.choice(candid_nodes, p=choice_prob)
                
                pop.append(choiced_node-num_ele)
            else:
                candid_chain = [pop+[ele-num_ele] for ele in candid_nodes]
                condid_brance_info = []
                condid_visit_info = []
                for candid_chain_ele in candid_chain:
                    candid_dic_key = ''.join(str(candid_chain_ele_ele) 
                                               for candid_chain_ele_ele in candid_chain_ele)
                    if memory_branch.get(candid_dic_key) != None:
                        condid_brance_info.append(memory_branch[candid_dic_key])
                    else:
                        condid_brance_info.append(0)
                    
                    if memory_visit.get(candid_dic_key) != None:
                        condid_visit_info.append(memory_visit[candid_dic_key])
                    else:
                        condid_visit_info.append(0)
                condid_brance_info = np.array(condid_brance_info)
                condid_visit_info = np.array(condid_visit_info)
                parent_dic_key = ''.join(str(pop_ele) for pop_ele in pop)
                parent_visit_info = memory_visit[parent_dic_key]
                
                
                choice_prob = getChoiceProb(condid_brance_info, condid_visit_info, parent_visit_info, 
                                            temperature, 1-len(pops)/pop_num)
                choiced_node = np.random.choice(candid_nodes, p=choice_prob)
                
                pop.append(choiced_node-num_ele)
            dic_key = ''.join(str(pop_ele) for pop_ele in pop)
            if memory_visit.get(dic_key) != None:
                memory_visit[dic_key] += 1
            else:
                memory_visit[dic_key] = 1
            candid_nodes = candidate.get_can_do_task(pop)
            flag = 0
            candid_nodes = (np.array(candid_nodes)+num_ele).tolist()
        pops.append(pop)
        if last_unique_num != np.unique(pops, axis=0).shape[0]:
            last_unique_num += 1
            for index_unique_pop in range(len(pop)):
                branch_key = ''.join(str(unique_pop_ele) for unique_pop_ele in pop[:index_unique_pop+1])
                if memory_branch.get(branch_key) != None:
                    memory_branch[branch_key] += 1
                else:
                    memory_branch[branch_key] = 1
    return np.array(pops)-1


def newPop(localElite_, popNum_, pop_s_, decisionD_, objectionD_, flag):
    localOptimalPopIndex = np.argwhere(localElite_==0).reshape(-1)
    if localOptimalPopIndex.shape[0]<=popNum_:
        return pop_s_[np.argsort(localElite)[0:popNum_]]
    else:
        ps = copy.deepcopy(pop_s_[localOptimalPopIndex])
        dD = copy.deepcopy(decisionD_[localOptimalPopIndex])
        oD = copy.deepcopy(objectionD_[localOptimalPopIndex])
        if flag==1:
            return ps[crowdDistance1(dD, oD)[0:popNum_]]
        else:
            decisionObjectionDistances = np.concatenate((dD[...,np.newaxis], oD[...,np.newaxis]),axis=1)
            rank, f = fast_non_dominated_sort(-1*decisionObjectionDistances)
            counter = 0
            for ii, ele in enumerate(f):
                counter = counter+len(ele)
                if counter>popNum_:
                    break
            cut_position = copy.deepcopy(ii)

            P = []
            if cut_position!=0:
                for iii in range(cut_position):
                    P = P+f[iii]
            supplement = traditionCrowdDistance(decisionObjectionDistances[f[cut_position]])[1][0:popNum_-len(P)].tolist()
            supplement = traditionCrowdDistance(decisionObjectionDistances[f[cut_position]])[1][0:popNum_-len(P)].tolist()
            supplement = np.array(f[cut_position])[supplement].tolist()
            return ps[P+supplement]
        
        
# straight line P8
constr_mat = np.array([[0,1,1,0,1,0,0,0],
                       [0,0,0,0,0,0,0,1],
                       [0,0,0,0,0,1,0,0],
                       [0,0,0,0,0,0,0,0],
                       [0,0,0,0,0,0,0,1],
                       [0,0,0,0,0,0,0,1],
                       [0,0,0,1,0,0,0,0],
                       [0,0,0,0,0,0,1,0]])
partNum = 8    
CT = 40
time = np.array([14, 10, 12, 18, 23, 16, 20, 14])
danger = np.array([0, 0, 0, 0, 0, 0, 1, 0 ])
need = np.array([360,500,620,480,540, 750, 295,360])
maskSide = np.ones(partNum) # indicates the form of production organization


# straight line P10
time_change = 0
constr_mat = np.array([[0,1,1,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,1,0,0], 
                       [0,0,0,0,0,0,1,0,0,0], 
                       [0,0,0,0,0,0,1,0,0,0], 
                       [0,0,0,0,0,0,0,1,0,0], 
                       [0,1,1,0,0,0,0,0,0,0], 
                       [0,1,1,0,0,0,0,0,0,0], 
                       [0,1,1,0,0,0,0,0,0,0]])
partNum = 10
CT = 40
time = np.array([14,10,12,17,23,14,19,36,14,10])
danger = np.array([0,0,0,0,0,0,1,0,0,0])
need = np.array([0,500,0,0,0,750,295,0,360,0])
maskSide = np.ones(partNum) # indicates the form of production organization


# straight line phone                
constr_mat = np.array([[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], 
                       [0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], 
                       [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])
partNum = 25      
CT = 18          
time = np.array([3,2,3,10,10,15,15,15,15,2,2,2,2,2,2,2,2,3,18,5,1,5,15,2,2])
danger = np.array([1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1])
need = np.array([4,7,1,1,1,1,1,1,1,2,1,4,1,1,1,1,2,2,8,1,4,6,7,1,4])
maskSide = np.ones(partNum) # indicates the form of production organization


def levenshteinDistance(str1, str2):
    matrix = [[ i + j for j in range(len(str2) + 1)] for i in range(len(str1) + 1)]
 
    for i in range(1, len(str1)+1):
        for j in range(1, len(str2)+1):
            if(str1[i-1] == str2[j-1]):
                d = 0
            else:
                d = 1
            matrix[i][j] = min(matrix[i-1][j]+1, matrix[i][j-1]+1, matrix[i-1][j-1]+d)
    return matrix[len(str1)][len(str2)]

def crowdDistance1(decisionDistance, objectionDistance):
    equilibrium = []
    for i in range(decisionDistance.shape[0]):
        equilibrium.append(min(decisionDistance[i], objectionDistance[i]))
    equilibrium = np.array(equilibrium)
    return np.flip(np.argsort(equilibrium))

def traditionCrowdDistance(popf):
    I = popf.shape[0]
    popDistance = np.zeros(shape=(I))
    for M in range(popf.shape[1]):
        popIndex = np.argsort(popf[:,M])
        popDistance[popIndex[0]] = 9999999999999999999999
        popDistance[popIndex[-1]] = 9999999999999999999999
        for i in range(1, I-1):
            popDistance[popIndex[i]] = popDistance[popIndex[i]] + (popf[:,M][popIndex[i+1]] - popf[:,M][popIndex[i-1]])/(popf[:,M][popIndex[-1]] - popf[:,M][popIndex[0]] + 0.000000001)
    return popDistance

def dominatedLevelDistance(popf):
    dominatedLevelDistanceM = np.zeros(shape=(popf.shape[0],popf.shape[0]))
    for i in range(popf.shape[0]):
        for j in range(popf.shape[0]):
            dominatedLevelDistanceM[i][j] = compare(popf[i], popf[j])
    dominatedLevelDistanceM[dominatedLevelDistanceM>=0]=0
    dominatedLevelDistanceM[dominatedLevelDistanceM<0]=1
    return dominatedLevelDistanceM

def objectionDomainDistance(popf):
    pop_f0 = np.expand_dims(popf,axis=0)
    pop_f0= np.broadcast_to(pop_f0,[int(popf.shape[0]), int(popf.shape[0]), int(popf.shape[1])])
    pop_f1 = np.expand_dims(popf,axis=1)
    pop_f1=np.broadcast_to(pop_f1,[int(popf.shape[0]), int(popf.shape[0]), int(popf.shape[1])])
    return np.sum(np.square(pop_f0 - pop_f1), axis=2)


def conbineFeature(F):
    return np.concatenate([ele[...,np.newaxis] for ele in F],axis=1)

def taskDistance(feature):
    feature_f0 = np.expand_dims(feature,axis=0)
    feature_f0= np.broadcast_to(feature_f0,[int(feature.shape[0]), int(feature.shape[0]), int(feature.shape[1])])
    feature_f1 = np.expand_dims(feature,axis=1)
    feature_f1=np.broadcast_to(feature_f1,[int(feature.shape[0]), int(feature.shape[0]), int(feature.shape[1])])
    recoder = np.sum(np.square(feature_f0 - feature_f1), axis=2)
    return recoder
def popDistance(pop1, pop2, recoder):
    combinePop = np.concatenate((pop1[...,np.newaxis], pop2[...,np.newaxis]),axis=1) # partNum*2
    D = []
    for i in range(combinePop.shape[0]):
        D.append(recoder[combinePop[i][0],combinePop[i][1]])
    return sum(D)
def KernelDistance(pops, recoder):
    distanceM = np.zeros(shape=(pops.shape[0],pops.shape[0]))
    for i in range(pops.shape[0]):
        for j in range(pops.shape[0]):
            distanceM[i][j] = popDistance(pops[i],pops[j],recoder)
    return distanceM

# harmonic mean
def targetCrowdDistance(objs):
    tcd = []
    for i in range(len(objs)):
        temps = []
        for j in range(len(objs)):
            if j != i:
                temps.append(1/sum((objs[i] - objs[j])**2))
        tcd.append((len(objs)-1)/sum(temps))
    return np.array(tcd)

def allot_part_2_station(part_order_, time_part_, beat_time_):
    workstation = []
    allot_part_2_workstation = []
    for ele in part_order_:
        workstation.append(ele)
        if sum(time_part_[workstation])>beat_time_:
            allot_part_2_workstation.append(workstation[:-1])
            workstation = []
            workstation.append(ele)
    allot_part_2_workstation.append(workstation)
    return allot_part_2_workstation


def fit_values(orders, time_part, beat, danger_coef, need_coef):
    num_gen = orders.shape[0]
    R = []
    for i in range(num_gen):
        order = orders[i]
        R.append(allot_part_2_station(part_order_=order, time_part_=time_part, beat_time_=beat))
    f1s = [len(R[i]) for i in range(num_gen)] 
    f2s = []
    for i in range(num_gen):
        re = []
        for j in range(f1s[i]):
            re.append(np.power(beat - sum(time_part[R[i][j]]), 2))
        f2s.append(sum(re))
    f3s = []
    for i in range(num_gen):
        re = []
        for j in range(len(orders[i])):
            re.append((j+1)*danger_coef[orders[i][j]])
        f3s.append(sum(re))
    f4s = []
    for i in range(num_gen):
        re = []
        for j in range(len(orders[i])):
            re.append((j+1)*need_coef[orders[i][j]])
        f4s.append(sum(re))
    return np.array([f1s,f2s,f4s,f3s]).T 
tempary = 0.01
sigma = 10000
popNum = 200
evaluateNum = 100
feature = conbineFeature([time,danger,need])
taskD = taskDistance(feature)
# random generation of parent and child merge individual
R_t = init(num_ele=partNum, pop_num=popNum, temperature = 1, 
                 prior_constrain = constr_mat, disassembly_side = maskSide)

for ii in range(evaluateNum):
    print('第',ii+1,'次进化')
    R_t = np.unique(R_t,axis=0)
    R_t_obj = fit_values(R_t, time, CT, danger, need)
    R_t_rank, R_t_f = fast_non_dominated_sort(R_t_obj)
    
    print(np.unique(R_t_obj[R_t_f[0]],axis=0))
    elit = []
    for cut, ele in enumerate(R_t_f):
        if len(elit + ele) <= popNum:
            elit = elit + ele
        else:
            break
    candid_pop = np.array(R_t_f[cut])
    decisionD = KernelDistance(R_t[candid_pop], taskD)
    R_t_decision_crowd = np.sum(decisionD,axis=1)
    R_t_target_crowd = targetCrowdDistance(R_t_obj[candid_pop])
    candid_obj = np.concatenate((R_t_decision_crowd[np.newaxis,...], R_t_target_crowd[np.newaxis,...])).T
    candid_obj = -1*candid_obj
    candid_rank, candid_f = fast_non_dominated_sort(candid_obj)
    candid_crowd = traditionCrowdDistance(candid_obj)
    need_supplement_num = popNum-len(elit)
    candid_elit = []
    for candid_cut, ele in enumerate(candid_f):
        if len(candid_elit + ele) <= need_supplement_num:
            candid_elit = candid_elit + ele
        else:
            break
    candid_pop_ = np.array(candid_f[candid_cut])
    candid_pop_crowd_ = candid_crowd[candid_pop_]

    need_supplement_num_ = need_supplement_num-len(candid_elit)
    candid_elit = candid_elit + candid_pop_[np.argsort(candid_pop_crowd_)[::-1][:need_supplement_num_]].tolist()
    elit = elit + candid_pop[candid_elit].tolist()
    P_t = R_t[elit]
    if ii<evaluateNum-1:
        child = []
        while True:
            if len(child)==popNum:
                break
            else:
                while True:
                    a1 = np.random.randint(0, popNum)
                    b1 = np.random.randint(0, popNum)
                    if a1!=b1:
                        break
                child.append(crossover(P_t[a1], P_t[b1], partNum, constr_mat, alpha=0.85))
        child = np.array(child)
        R_t = np.concatenate([P_t, child])

第 0 次进化
[[ 10  81 946  85]
 [ 10  97 947  84]
 [ 10  99 930  82]
 [ 10 103 940  80]
 [ 10 107 929  78]
 [ 10 125 900  85]
 [ 10 131 928  82]
 [ 10 135 926  78]
 [ 10 139 918  78]
 [ 10 167 915  80]
 [ 10 193 913  81]
 [ 11 305 927  77]
 [ 11 315 910  79]
 [ 11 335 922  77]
 [ 12 529 913  77]
 [ 12 563 896  78]]
第 1 次进化
[[ 10  81 946  85]
 [ 10  89 944  81]
 [ 10  99 930  82]
 [ 10 103 940  80]
 [ 10 107 910  74]
 [ 10 125 900  85]
 [ 10 193 899  79]
 [ 11 313 872  81]
 [ 11 355 901  75]
 [ 12 563 896  78]]
第 2 次进化


C:\Users\24545\.conda\envs\TF2.4\lib\site-packages\ipykernel_launcher.py:418: RuntimeWarning: divide by zero encountered in int_scalars


[[ 10  81 929  81]
 [ 10  97 932  80]
 [ 10 107 910  74]
 [ 10 125 900  85]
 [ 10 129 886  85]
 [ 10 171 905  74]
 [ 10 173 899  79]
 [ 11 313 872  81]
 [ 11 323 904  76]
 [ 11 355 900  75]
 [ 12 563 896  78]]
第 3 次进化
[[ 10  81 923  81]
 [ 10  81 931  80]
 [ 10  99 921  81]
 [ 10  99 929  78]
 [ 10 105 926  78]
 [ 10 107 910  74]
 [ 10 119 880  84]
 [ 10 139 902  76]
 [ 10 171 905  74]
 [ 10 173 899  79]
 [ 11 301 866  80]
 [ 11 331 898  77]
 [ 11 355 900  75]
 [ 12 475 863  77]
 [ 12 529 883  74]]
第 4 次进化
[[ 10  81 919  79]
 [ 10  89 914  80]
 [ 10  99 929  78]
 [ 10 105 914  75]
 [ 10 107 909  90]
 [ 10 107 910  74]
 [ 10 119 880  84]
 [ 10 123 897  75]
 [ 10 127 876  82]
 [ 10 171 905  74]
 [ 11 301 866  80]
 [ 11 319 895  76]
 [ 11 335 894  74]
 [ 12 475 863  77]
 [ 12 529 883  74]]
第 5 次进化
[[ 10  81 919  79]
 [ 10  89 914  80]
 [ 10  89 918  78]
 [ 10 105 914  75]
 [ 10 107 909  90]
 [ 10 107 910  74]
 [ 10 119 879  84]
 [ 10 119 884  83]
 [ 10 123 897  75]
 [ 10 123 902  73]
 [ 1

[[  9   9 852  78]
 [  9   9 875  77]
 [  9  13 849  79]
 [ 10  97 899  76]
 [ 10 105 901  75]
 [ 10 107 904  74]
 [ 10 111 889  74]
 [ 10 113 884  74]
 [ 10 113 886  73]
 [ 10 115 857  77]
 [ 10 119 853  77]
 [ 10 123 880  76]
 [ 10 123 884  73]
 [ 10 123 886  72]
 [ 10 131 829  78]
 [ 10 135 850  77]
 [ 10 153 879  72]
 [ 10 155 848  77]
 [ 10 155 872  71]
 [ 10 159 847  77]
 [ 10 173 835  76]
 [ 10 173 839  75]
 [ 11 267 835  75]
 [ 11 299 829  74]
 [ 11 319 874  70]
 [ 11 345 871  70]
 [ 11 347 826  74]
 [ 11 347 827  73]
 [ 11 347 870  69]
 [ 12 521 820  74]
 [ 12 579 816  74]]
第 21 次进化
[[  9   9 852  78]
 [  9   9 875  77]
 [  9  13 849  79]
 [ 10  97 899  76]
 [ 10 105 901  75]
 [ 10 107 904  74]
 [ 10 111 889  74]
 [ 10 113 884  74]
 [ 10 113 886  73]
 [ 10 115 857  77]
 [ 10 119 853  77]
 [ 10 123 880  76]
 [ 10 123 884  73]
 [ 10 123 886  72]
 [ 10 131 829  78]
 [ 10 135 850  77]
 [ 10 141 841  77]
 [ 10 153 879  72]
 [ 10 155 872  71]
 [ 10 173 835  76]
 [ 10 173 839  75]
 [

第 33 次进化
[[  9   9 833  78]
 [  9   9 874  77]
 [  9  13 848  77]
 [ 10  89 902  76]
 [ 10  97 899  76]
 [ 10  99 897  76]
 [ 10 103 854  76]
 [ 10 103 888  72]
 [ 10 105 885  71]
 [ 10 109 849  76]
 [ 10 113 884  74]
 [ 10 115 841  76]
 [ 10 123 833  76]
 [ 10 123 835  75]
 [ 10 123 884  73]
 [ 10 131 829  78]
 [ 10 131 831  77]
 [ 10 135 830  77]
 [ 10 141 822  79]
 [ 10 141 829  77]
 [ 10 141 831  76]
 [ 10 147 829  75]
 [ 10 153 873  73]
 [ 10 153 875  72]
 [ 10 155 825  77]
 [ 10 155 827  76]
 [ 10 155 870  72]
 [ 10 155 872  71]
 [ 10 161 826  75]
 [ 10 165 823  78]
 [ 10 171 815  76]
 [ 10 213 811  75]
 [ 10 217 825  74]
 [ 11 299 816  73]
 [ 11 319 874  70]
 [ 11 339 807  74]
 [ 11 345 870  70]
 [ 11 347 865  70]
 [ 11 347 867  69]
 [ 11 363 809  73]
 [ 11 387 809  72]
 [ 12 511 807  70]]
第 34 次进化
[[  9   9 833  78]
 [  9   9 874  77]
 [  9  13 848  77]
 [ 10  89 902  76]
 [ 10  97 899  76]
 [ 10  99 897  76]
 [ 10 103 854  76]
 [ 10 103 888  72]
 [ 10 105 885  71]
 [ 10 109 84

第 49 次进化
[[  9   9 815  76]
 [  9   9 817  75]
 [ 10 103 888  72]
 [ 10 105 885  71]
 [ 10 113 884  74]
 [ 10 123 884  73]
 [ 10 135 812  75]
 [ 10 135 814  74]
 [ 10 139 813  74]
 [ 10 153 873  73]
 [ 10 153 875  72]
 [ 10 155 870  72]
 [ 10 155 872  71]
 [ 10 199 808  74]
 [ 10 199 810  73]
 [ 11 319 804  73]
 [ 11 319 874  70]
 [ 11 327 809  72]
 [ 11 331 805  71]
 [ 11 345 870  70]
 [ 11 347 865  70]
 [ 11 347 867  69]
 [ 11 385 803  72]
 [ 12 511 799  71]
 [ 12 511 801  70]]
第 50 次进化
[[  9   9 815  76]
 [  9   9 817  75]
 [ 10 103 888  72]
 [ 10 105 885  71]
 [ 10 113 884  74]
 [ 10 123 884  73]
 [ 10 135 812  75]
 [ 10 135 814  74]
 [ 10 139 813  74]
 [ 10 153 873  73]
 [ 10 153 875  72]
 [ 10 155 870  72]
 [ 10 155 872  71]
 [ 10 199 808  74]
 [ 10 199 810  73]
 [ 11 319 804  73]
 [ 11 319 874  70]
 [ 11 327 809  72]
 [ 11 331 805  71]
 [ 11 345 870  70]
 [ 11 347 865  70]
 [ 11 347 867  69]
 [ 11 385 803  72]
 [ 12 511 799  71]
 [ 12 511 801  70]]
第 51 次进化
[[  9   9 815  76]
 [

第 66 次进化
[[  9   9 815  76]
 [  9   9 817  75]
 [  9  13 811  75]
 [ 10 103 888  72]
 [ 10 105 885  71]
 [ 10 113 884  74]
 [ 10 123 884  73]
 [ 10 135 814  74]
 [ 10 139 808  74]
 [ 10 139 810  73]
 [ 10 155 870  72]
 [ 10 155 872  71]
 [ 10 193 807  74]
 [ 10 193 809  73]
 [ 11 303 812  72]
 [ 11 319 804  73]
 [ 11 319 806  72]
 [ 11 319 874  70]
 [ 11 331 803  72]
 [ 11 331 805  71]
 [ 11 347 865  70]
 [ 11 347 867  69]
 [ 11 385 802  72]
 [ 11 385 804  71]
 [ 12 511 799  71]
 [ 12 511 801  70]]
第 67 次进化
[[  9   9 815  76]
 [  9   9 817  75]
 [  9  13 811  75]
 [ 10 103 888  72]
 [ 10 105 885  71]
 [ 10 113 884  74]
 [ 10 135 814  74]
 [ 10 139 808  74]
 [ 10 139 810  73]
 [ 10 155 870  72]
 [ 10 155 872  71]
 [ 10 193 807  74]
 [ 10 193 809  73]
 [ 11 303 822  72]
 [ 11 319 804  73]
 [ 11 319 806  72]
 [ 11 319 874  70]
 [ 11 331 803  72]
 [ 11 331 805  71]
 [ 11 347 865  70]
 [ 11 347 867  69]
 [ 11 385 802  72]
 [ 11 385 804  71]
 [ 12 511 799  71]
 [ 12 511 801  70]]
第 68 次进化
[[

第 84 次进化
[[  9   9 815  76]
 [  9   9 817  75]
 [  9  13 811  75]
 [  9  13 813  74]
 [ 10 103 888  72]
 [ 10 105 885  71]
 [ 10 139 808  74]
 [ 10 139 810  73]
 [ 10 153 875  72]
 [ 10 155 870  72]
 [ 10 155 872  71]
 [ 10 193 809  73]
 [ 10 205 806  73]
 [ 11 319 806  72]
 [ 11 319 874  70]
 [ 11 331 803  72]
 [ 11 331 806  71]
 [ 11 345 870  70]
 [ 11 347 865  70]
 [ 11 347 867  69]
 [ 11 385 802  72]
 [ 12 511 799  71]
 [ 12 511 801  70]]
第 85 次进化
[[  9   9 815  76]
 [  9   9 817  75]
 [  9  13 811  75]
 [  9  13 813  74]
 [ 10 103 888  72]
 [ 10 105 885  71]
 [ 10 139 808  74]
 [ 10 139 810  73]
 [ 10 153 875  72]
 [ 10 155 870  72]
 [ 10 155 872  71]
 [ 10 193 809  73]
 [ 10 205 806  73]
 [ 10 205 808  72]
 [ 11 319 806  72]
 [ 11 319 874  70]
 [ 11 331 803  72]
 [ 11 331 806  71]
 [ 11 345 870  70]
 [ 11 385 802  72]
 [ 12 511 799  71]
 [ 12 511 801  70]]
第 86 次进化
[[  9   9 815  76]
 [  9   9 817  75]
 [  9  13 813  74]
 [ 10 103 888  72]
 [ 10 105 885  71]
 [ 10 139 808  74]
 [